# Zadanie 1 – analýza tweetov  
Odovzdanie do 10.10.2021 23:59 – máte na to presne 2 týždne – dostanete za to 7,5 boda. 
 
Prvé zadanie je zamerané na roztriedenie tweetov medzi rôzne konšpiračné teórie uvedené 
pod zadaním. Programovať môžete v hocijakom jazyku a váš zdroják sa odovzdáva to IS, no 
rovnako MUSÍ byť zavesený na vašom githube – v dokumente na začiatku uveďte vždy linku 
na projekt. Ak nebude GitHub, nebudú body. Okrem zdrojáku odovzdávate aj dokument kde 
budú screenshoty vašich výsledkov, výsledky ako text a grafy. Môžete napísať aj nejaké teplé 
slovko k tomu, nech sa pobavíme. 
 
Vašou úlohou je teda: 
 
1. Importovať dataset do PostgreSQL 13+: 
https://drive.google.com/drive/folders/1_crPerzWU2Nzc4mR5k6xuGqbp0fJDvsY?usp=sharing 
 
2. Vypočítať sentiment pomocou VADER lexikónu 
https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664 pre 
tweety, ktoré obsahujú nasledovné hashtagy: #DeepstateVirus #DeepStateVaccine 
#DeepStateFauci #QAnon #Agenda21 #CCPVirus #ClimateChangeHoax 
#GlobalWarmingHoax #ChinaLiedPeopleDied #SorosVirus #5GCoronavirus #MAGA 
#WWG1WGA #Chemtrails #flatEarth #MoonLandingHoax #moonhoax #illuminati 
#pizzaGateIsReal #PedoGateIsReal #911truth #911insidejob #reptilians 
Pri výpočte nezohľadňujte (ignorujte) emotikony ani hashtagy ani mentiony v texte. 
 
3. Roztriediť vyfiltrované tweety z predošlého zadania medzi konšpiračné teórie – 
spravte si na to novú tabuľku a mapovanie – nech je jasné ktorý tweet patrí ktorej 
konšpiračnej teórii.  
 
4. Vypočítajte pomer extrémnych a neutrálnych sentimentov tweetov pre konšpiračné 
teórie po týždňoch a zistite, či daná konšpiračná teória rastie alebo upadá v čase. 
Výstup vizualizujte v grafe. Rovnako uveďte aj absolútne čísla: tweet_count, 
tweet_extreme_count, tweet_neutral_count v tabuľke pre každý týždeň. Za extrém 
považujeme keď je compound väčší ako 0,5 alebo menší ako -0,5. 
 
5. Nájdite TOP10 account-ov ktoré sú najaktívnejšie v každej konšpiračnej teórii 
s extrémnym sentimentom a ukážte ich v tabuľke: id, name, screen_name a 
tweet_count. 
 
6. Nájdite TOP10 najčastejšie používaných hashtagov pre každú konšpiračnú teóriu 
z tweetov s extrémnym sentimentom, vypíšte aj počet. 
 
V úvahu berte nasledovné konšpiračné teórie a ich hashtagy.

## Database structure

![DB structure](Zadanie1-TweetsDB.jpeg)

# === Solution ===
## by Bc. Juraj Baráth

## Task 1 - Import dataset to PostgreSQL 13+

#### Step 1: Install postgis extension
`apt install -y postgis postgresql-postgis`

#### Step 2: Create new empty database
`su postgres`

`psql`
```
CREATE DATABASE tweets;
quit;
```
`logout`

#### Step 3: Import the database 
`screen -dmS pdt pg_restore -v -d tweets tweets.backup`

## Task 2 - Sentiment Calculation using VADER

#### Step 1: Grant read only access to a new database user
`su postgres`

`psql -d tweets`

```
CREATE USER pdt with password '9kRoaGZN5EcgTeDM';
GRANT CONNECT ON DATABASE tweets TO pdt;
GRANT USAGE ON SCHEMA public TO pdt;
GRANT SELECT ON ALL TABLES IN SCHEMA public TO pdt;
quit;
```

`logout`

#### Step 2: Install pip
`apt install -y python3-pip`

#### Step 3: Install postgresql connector (psycopg2) dependencies
`apt-get install -y libpq-dev`

#### Step 4: Install postgresql connector (psycopg2)
`pip install psycopg2`

#### Step 5: Initialize Vader

In [1]:
# Download vader_lexicon
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Create Sentiment Intensity Analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

#### Step 6: Initialize database connection

In [3]:
# Connect to database
import psycopg2
def connect():
    conn = psycopg2.connect(
    host="localhost",
    database="tweets",
    user="pdt",
    password="9kRoaGZN5EcgTeDM")
    return conn.cursor()

#### Step 7: Describe available tables

In [4]:
# List available tables
with connect() as cur:
    cur.execute("select tablename from pg_tables where schemaname='public'")
    tables = [a[0] for a in cur.fetchall()]
    tables.sort()
    print('Available tables:',tables)

Available tables: ['accounts', 'countries', 'hashtags', 'spatial_ref_sys', 'tweet_hashtags', 'tweet_mentions', 'tweets']


In [5]:
# Describe tables
with connect() as cur:
    for t in tables:
        print('Table '+t+':')
        cur.execute("select column_name, data_type, character_maximum_length, column_default, is_nullable " +
            "from INFORMATION_SCHEMA.COLUMNS where table_name = '" + t + "'")
        for r in cur.fetchall():
            print(r)
        print('')
    

Table accounts:
('id', 'bigint', None, "nextval('accounts_id_seq'::regclass)", 'NO')
('screen_name', 'character varying', 200, None, 'YES')
('name', 'character varying', 200, None, 'YES')
('description', 'text', None, None, 'YES')
('followers_count', 'integer', None, None, 'YES')
('friends_count', 'integer', None, None, 'YES')
('statuses_count', 'integer', None, None, 'YES')

Table countries:
('id', 'integer', None, "nextval('countries_id_seq'::regclass)", 'NO')
('code', 'character varying', 2, None, 'YES')
('name', 'character varying', 200, None, 'YES')

Table hashtags:
('id', 'integer', None, "nextval('hashtags_id_seq'::regclass)", 'NO')
('value', 'text', None, None, 'NO')

Table spatial_ref_sys:
('srid', 'integer', None, None, 'NO')
('auth_name', 'character varying', 256, None, 'YES')
('auth_srid', 'integer', None, None, 'YES')
('srtext', 'character varying', 2048, None, 'YES')
('proj4text', 'character varying', 2048, None, 'YES')

Table tweet_hashtags:
('id', 'integer', None, "next

In [6]:
# Show sample data
for t in tables:
    print('Table '+t+':')
    with connect() as cur:
        cur.execute('SELECT * FROM '+t+' LIMIT 10')
        for r in cur.fetchall():
            print(r)
            if len(str(r)) > 50:
                print('')
        print('')

Table accounts:
(280887543, 'carol_cabural', 'Caroline Cabural ♔', "Always look at things from the other person's perspective.\n-Dale Carnegie \nGarde Manger Chef| Hospitality Industry | Filipino", 657, 1482, 4650)

(519245371, 'strong_sue', 'Sue Strong 🇺🇸🇦🇺🇨🇦🇳🇿🇬🇧 #BidenHarris2020', '#HearHim, #ChurchOfJesusChrist, #StandUpRepublic, #TeamPatriot, #CountryOverParty, #ETTD, #Cats, #MS, TSB, GTT, #PAM, #PAMFAM #TeamJoe', 1469, 2636, 53614)

(761224507, 'Chn_analysis', 'Paul Harding', '', 150, 357, 4993)

(905154964829462528, 'thespybrief', 'The Spy Brief', None, None, None, None)

(707774148072882176, 'hiemakene', 'GoneFishing', None, None, None, None)

(1133475349285691393, 'KtyWise', 'Kty Wise', '', 161, 241, 21786)

(532977876, 'Adrienne711', 'NeverGiveUp ⭐⭐⭐ 🐸', 'Trump2020 WWG1WGA  1A 2A AMERICA FIRST -  I ♥ God Country  & our President Donald J Trump   UnitedNotDivided WWG1WGA\nShadilay!', 7335, 7033, 132331)

(949056207448432640, 'MBJ8388', "MBJ👩\u200d💻NoFilters💥Q'Nited✨", None, Non

#### Step 8: Create hashtag -> conspiracy mapping

In [7]:
# Create list of conspiracy theories
conspiracies=('Deep State','Qanon','New world order','The virus escaped from a Chinese lab','Global Warming is HOAX','COVID19 and microchipping',
              'COVID19 is preaded by 5G','Moon landing is fake','9/11 was inside job',
              'Pizzagate conspiracy theory','Chemtrails',
              'Flat Earth','Illuminati','Reptilian conspiracy theory')

# Map hashtags to the list of conspiracy theories
hashtag_conmap={'#DeepstateVirus':0,'#DeepStateVaccine':0,'#DeepStateFauci':0,
          '#QAnon':1,'#MAGA':1,'#WWG1WGA':1,
          '#Agenda21':2,
          '#ChinaLiedPeopleDied':3,'#CCPVirus':3,
          '#ClimateChangeHoax':4, '#GlobalWarmingHoax':4,
          '#SorosVirus':5, # '#BillGAtes':5
          '#5GCoronavirus':6,
          '#MoonLandingHoax':7,'#moonhoax':7,
          '#911truth':8,'#911insidejob':8,
          '#pizzaGateIsReal':9,'#PedoGateIsReal':9,
          '#Chemtrails':10,
          '#flatEarth':11,
          '#illuminati':12,
          '#reptilians':13}

# Print the generated mappings in alphabetical order
hashtags = list(hashtag_conmap.keys())
hashtags.sort(key = lambda e: e.upper())
for t in hashtags:
    print(t,' - ',conspiracies[hashtag_conmap[t]])

#5GCoronavirus  -  COVID19 is preaded by 5G
#911insidejob  -  9/11 was inside job
#911truth  -  9/11 was inside job
#Agenda21  -  New world order
#CCPVirus  -  The virus escaped from a Chinese lab
#Chemtrails  -  Chemtrails
#ChinaLiedPeopleDied  -  The virus escaped from a Chinese lab
#ClimateChangeHoax  -  Global Warming is HOAX
#DeepStateFauci  -  Deep State
#DeepStateVaccine  -  Deep State
#DeepstateVirus  -  Deep State
#flatEarth  -  Flat Earth
#GlobalWarmingHoax  -  Global Warming is HOAX
#illuminati  -  Illuminati
#MAGA  -  Qanon
#moonhoax  -  Moon landing is fake
#MoonLandingHoax  -  Moon landing is fake
#PedoGateIsReal  -  Pizzagate conspiracy theory
#pizzaGateIsReal  -  Pizzagate conspiracy theory
#QAnon  -  Qanon
#reptilians  -  Reptilian conspiracy theory
#SorosVirus  -  COVID19 and microchipping
#WWG1WGA  -  Qanon


#### Step 9: Fetch hashtag -> hashtag-ids mapping

In [8]:
hashtag_idmap={}
for t in hashtags:
    # Since the dataset does not respect our case conventions, we are doing a case insensitive search
    # We are also looking for hashtags with & without the # prefix
    with connect() as cur:
        cur.execute("SELECT id FROM hashtags WHERE LOWER(value) = '" + t[1:].lower() + "' "+
                "OR LOWER(value) = '" + t[1:].lower() + "'")
        rows = cur.fetchall()
    
        # Save the results to a list of ids, map the list to the given hashtag
        ids=list(map(lambda r: r[0], rows))
        hashtag_idmap[t] = ids
        print(t,'->',ids)

#5GCoronavirus -> [15552438, 19005036, 34619065, 35538816]
#911insidejob -> [16074404]
#911truth -> [2525313, 7014593]
#Agenda21 -> [66445, 363281, 111971]
#CCPVirus -> [34584873, 10905016, 3073063, 29467281, 31317877, 18509464, 1137275, 34691278, 1379885]
#Chemtrails -> [15171991, 33792, 905802, 149294]
#ChinaLiedPeopleDied -> [16685336, 23538315, 26615200, 23714496, 25456041, 5112446, 27219903, 36886795]
#ClimateChangeHoax -> [14157066, 159068, 10090147]
#DeepStateFauci -> [16542067, 15870712]
#DeepStateVaccine -> [17407797]
#DeepstateVirus -> [7074589, 18007452]
#flatEarth -> [20256863, 18277418, 4209470, 3436024, 7529160]
#GlobalWarmingHoax -> [3486140, 4673374]
#illuminati -> [875154, 3178025, 967857, 16238392]
#MAGA -> [17899079, 15894243, 19261059, 19898566, 88504, 24818, 19469799, 14415]
#moonhoax -> [18587371]
#MoonLandingHoax -> [19604351]
#PedoGateIsReal -> [15550095, 2741960, 15659618, 15268807, 6005507, 7832331]
#pizzaGateIsReal -> [15188058, 18874438, 15673159, 17812154, 

#### Step 10: Fetch hashtag -> tweet-id mapping

In [9]:
hashtag_twmap={}
for t in hashtags:
    twids = set()
    for hid in hashtag_idmap[t]:
        with connect() as cur:
            cur.execute("SELECT tweet_id FROM tweet_hashtags WHERE hashtag_id = "+str(hid))
            rows = cur.fetchall()
            for r in rows:
                twids.add(r[0])
    hashtag_twmap[t]=twids
    print(t,'-> mapped',len(twids),'tweets')

#5GCoronavirus -> mapped 407 tweets
#911insidejob -> mapped 4 tweets
#911truth -> mapped 139 tweets
#Agenda21 -> mapped 2297 tweets
#CCPVirus -> mapped 5024 tweets
#Chemtrails -> mapped 1044 tweets
#ChinaLiedPeopleDied -> mapped 3261 tweets
#ClimateChangeHoax -> mapped 53 tweets
#DeepStateFauci -> mapped 183 tweets
#DeepStateVaccine -> mapped 14 tweets
#DeepstateVirus -> mapped 518 tweets
#flatEarth -> mapped 340 tweets
#GlobalWarmingHoax -> mapped 5 tweets
#illuminati -> mapped 2172 tweets
#MAGA -> mapped 207933 tweets
#moonhoax -> mapped 11 tweets
#MoonLandingHoax -> mapped 6 tweets
#PedoGateIsReal -> mapped 342 tweets
#pizzaGateIsReal -> mapped 3253 tweets
#QAnon -> mapped 2126183 tweets
#reptilians -> mapped 64 tweets
#SorosVirus -> mapped 7 tweets
#WWG1WGA -> mapped 697307 tweets


#### Step 11: Create the list of analysable tweets

In [10]:
tweet_ids=set()
for t in hashtags:
    tweet_ids.update(list(hashtag_twmap[t]))
tweet_ids=list(tweet_ids)
tweet_count=len(tweet_ids)
print('Found',tweet_count,'analysable tweets')

Found 2144772 analysable tweets


#### Step 12: Fetch tweets (aync)

In [ ]:
from time import sleep
from concurrent.futures import ThreadPoolExecutor
tweets={}

def fetch_tweets(fr,to):
    with connect() as cur:
        cur.execute("SELECT id,content FROM tweets WHERE id IN (" + str(tweet_ids[fr:to])[1:-1]+")")
        rows=cur.fetchall()
        for r in rows:
            tweets[r[0]]=r[1]

with ThreadPoolExecutor(max_workers=8) as e:
    for i in range(0,tweet_count,100000):
        to=min(tweet_count,i+100000)
        e.submit(fetch_tweets,i,to)
e.shutdown(wait=True)
print('Fetched',len(tweets),'tweets')

#### Step 13: Remove emoticons, hashtags, mentions from tweets

In [12]:
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

mention_hashtag_pattern=re.compile("[#@][A-Za-z0-9_]+")
i=0
for tid in tweet_ids:
    i+=1
    noemoji=emoji_pattern.sub(r'',tweets[tid])
    tweets[tid]=mention_hashtag_pattern.sub(r'',noemoji)
    if i%100000==0:
        print('Processed',i,'tweets')
print('Processed all tweets')

Processed 100000 tweets
Processed 200000 tweets
Processed 300000 tweets
Processed 400000 tweets
Processed 500000 tweets
Processed 600000 tweets
Processed 700000 tweets
Processed 800000 tweets
Processed 900000 tweets
Processed 1000000 tweets
Processed 1100000 tweets
Processed 1200000 tweets
Processed 1300000 tweets
Processed 1400000 tweets
Processed 1500000 tweets
Processed 1600000 tweets
Processed 1700000 tweets
Processed 1800000 tweets
Processed 1900000 tweets
Processed 2000000 tweets
Processed 2100000 tweets
Processed all tweets


#### Step 14: Calculate the sentiment of tweets (in async)

In [ ]:
tweet_sen={}
def calc_sen(fr,to):
    for i in range(fr,to):
        tid=tweet_ids[i]
        tweet_sen[tid]=sid.polarity_scores(tweets[tid])

print('Starting processor tasks...')
with ThreadPoolExecutor(max_workers=64) as e:
    for i in range(0,tweet_count,10000):
        to=min(tweet_count,i+10000)
        e.submit(calc_sen,i,to)
    print('Started processor tasks, waiting for completion...')
    while 1:
        sleep(2)
        processed=len(tweet_sen)
        print('Processed',processed,'tweets')
        if processed==tweet_count:
            e.shutdown(wait=True)

Starting processor tasks...
Started processor tasks, waiting for completion...
Processed 19910 tweets
Processed 31488 tweets
Processed 35989 tweets
Processed 39559 tweets
Processed 43135 tweets
Processed 46998 tweets
Processed 50653 tweets
Processed 53947 tweets
Processed 62924 tweets
Processed 66230 tweets
Processed 70982 tweets
Processed 74490 tweets
Processed 79378 tweets
Processed 83132 tweets
Processed 88346 tweets
Processed 93540 tweets
Processed 96796 tweets
Processed 99522 tweets
Processed 103638 tweets
Processed 106861 tweets
Processed 112883 tweets
Processed 116099 tweets
Processed 120265 tweets
Processed 123287 tweets
Processed 129930 tweets
Processed 134664 tweets
Processed 142849 tweets
Processed 147976 tweets
Processed 153381 tweets
Processed 156310 tweets
Processed 162235 tweets


In [ ]:
print('asd')

## Task 3 - Sort tweets by conspiracy theories

## Task 4 - Calculate the ratio of tweets with extreme / neutral sentiment

## Task 5 - Find Top 10 most active accounts in all extreme sentiment conspiracy theories

## Task 6 - Find Top 10 most used hashtags for each exteme sentiment conspiracy theories 

## Deep State  
 
https://en.wikipedia.org/wiki/Deep_state 
 
A deep state is a type of governance made up of potentially secret and unauthorised 
networks of power operating independently of a state's political leadership in pursuit of 
their own agenda and goals. In popular usage, the term carries overwhelmingly negative 
connotations. The range of possible uses of the term is similar to that for shadow 
government. The expression state within a state is an older and similar concept. Historically, 
it designated a well-defined organization which seeks to function independently, whereas 
the deep state refers more to a hidden organization seeking to manipulate the public state. 
 
`#DeepstateVirus #DeepStateVaccine #DeepStateFauci`

## Qanon  
 
https://dennikn.sk/2474994/pripraveni-na-velke-prebudenie-zo-zakulisia-hnutia-qanon/ 
 
“The [QAnon] conspiracy involves a range of batshit theories but very basically, many 
QAnon supporters believe in a global conspiracy involving a ring of Satan-worshipping, child-
molesting criminals led by prominent Democrats that includes everyone from Hillary Clinton 
to Tom Hanks to a Mexican cement company [Cemex]. They also believe information about 
it is being leaked via cryptic posts on the internet by someone with very high Q-level 
security clearance—which sounds just as made-up as it definitely is.” 
 
`#QAnon #MAGA #WWG1WGA`

## New world order  
 
https://www.buzzfeed.com/joeydurso/coronavirus-agenda-21-5g

An Old Conspiracy Theory Known As "Agenda 21" Has Been Rebooted By The Coronavirus 
Pandemic. New World Order is here again. 
 
`#Agenda21`

## The virus escaped from a Chinese lab  
 
https://www.vox.com/22453571/lab-leak-covid-19-coronavirus-hypothesis-wuhan-virology-china 
 
Where did the virus that causes Covid-19 come from? It’s one of the most persistent 
mysteries of the pandemic. The debate about it among scientists, policymakers, journalists, 
amateur internet sleuths, and the general public has reignited with new revelations and 
new voices in the mix. We may never know for sure if the virus that causes Covid-19 leaked 
from a lab. But that won’t stop the debate. 
 
`#CCPVirus #ChinaLiedPeopleDied`

## Global Warming is HOAX  
 
https://www.hull.ac.uk/special/blog/why-people-still-believe-climate-change-is-fake-and-
why-we-know-theyre-wrong 
 
Why people still believe climate change is fake...

`#ClimateChangeHoax #GlobalWarmingHoax`

## COVID19 and microchipping  
 
https://www.politifact.com/factchecks/2020/may/28/facebook-posts/theres-no-plot-
microchip-people-during-covid-19-te/ 
 
There’s no evidence that Bill Gates and George Soros plan to microchip people being tested 
for coronavirus. 
 
`#SorosVirus #BillGAtes`

## COVID19 is preaded by 5G  
 
https://www.vox.com/recode/2020/4/24/21231085/coronavirus-5g-conspiracy-theory-
covid-facebook-youtube 
 
The first link John Gregory saw pushing a connection between 5G and the coronavirus 
pandemic was on a French conspiracy website called Les moutons enragés, which loosely 
translates as “The rabid sheep.” A January 20 post floated that the millimeter wave 
spectrum used by 5G technology and Covid-19, the disease caused by the novel coronavirus, 
could be related, pointing to reports about Wuhan installing 5G towers before the outbreak. 
Three months later, conspiracy theorists making similar claims were setting cellphone 
towers on fire in Europe. 
 
`#5GCoronavirus`

## Moon landing is fake  
 
https://www.space.com/apollo-11-moon-landing-hoax-believers.html 
 
Moon-Landing Hoax Still Lives On, 50 Years After Apollo 11. 
 
`#MoonLandingHoax #moonhoax`

## 9/11 was inside job  
 
https://www.bbc.com/news/58469600 
 
11 September 2001: The conspiracy theories still spreading after 20 years 
 
`#911truth #911insidejob`

## Pizzagate conspiracy theory 
 
(https://en.wikipedia.org/wiki/Pizzagate_conspiracy_theory 
 
In March 2016, the personal email account of John Podesta, Hillary Clinton's campaign chair, 
was hacked in a spear-phishing attack. WikiLeaks published his emails in November 2016. 
Proponents of the Pizzagate conspiracy theory falsely claimed the emails contained coded 
messages that connected several high-ranking Democratic Party officials and U.S. 
restaurants with an alleged human trafficking and child sex ring. One of the establishments 
allegedly involved was the Comet Ping Pong pizzeria in Washington, D.C.[6][7] 
 
`#pizzaGateIsReal #PedoGateIsReal`

## Chemtrails  
 
https://keith.seas.harvard.edu/chemtrails-conspiracy-theory 
 
Chemtrails refers to the theory that governments or other parties are engaged in a secret 
program to add toxic chemicals to the atmosphere from aircraft in a way that forms visible 
plumes in the sky, somewhat similar to contrails. Various different motivations for this 
alleged spraying are speculated, including sterilization, reduction of life expectancy, mind 
control or weather control. 
 
`#Chemtrails`

## Flat Earth  
 
https://edition.cnn.com/2019/11/16/us/flat-earth-conference-conspiracy-theories-scli-intl/index.html 
 
(CNN)"I don't want to be a flat Earther," David Weiss says, his voice weary as he reflects on 
his personal awakening. "Would you wake up in the morning and want everyone to think 
you're an idiot?" But Weiss is a flat Earther. Ever since he tried and failed to find proof of the 
Earth's curve four years ago, he's believed with an evident passion that our planet is both 
flat and stationary -- and it's turned his world upside down. 
 
`#flatEarth`

## Illuminati  
 
https://www.bbc.com/future/article/20170809-the-accidental-invention-of-the-illuminati-
conspiracy 
It’s the conspiracy theory to dwarf all conspiracy theories. A smorgasbord of every other 
intrigue under the sun, the Illuminati are the supposed overlords controlling the world’s 
affairs, operating secretly as they seek to establish a New World Order. 
 
`#illuminati`

## Reptilian conspiracy theory  
 
https://en.wikipedia.org/wiki/Reptilian_conspiracy_theory 
 
Reptilians – also called reptoids, lizard people, reptiloids, saurians, or draconians – are 
supposed reptilian humanoids, which play a prominent role in fantasy, science fiction, 
ufology, and conspiracy theories. The idea of reptilians was popularised by David Icke, a 
conspiracy theorist who claims shapeshifting reptilian aliens control Earth by taking on 
human form and gaining political power to manipulate human societies. Icke has stated on 
multiple occasions that many world leaders are, or are possessed by, so-called reptilians. 
 
`#reptilians`